In [341]:
pwd

'/home/jupyter/Spotify'

In [342]:
## Artists to compare
artists_for_analysis=['Megan Thee Stallion','Benee','Lil Nas X','BTS','Doja Cat','Olivia Rodrigo']

In [343]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [344]:
pip install statistics

Note: you may need to restart the kernel to use updated packages.


In [345]:
import spotipy
import statistics as st
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import base64
import datetime
from urllib.parse import urlencode
import requests

In [346]:
## SpotifyAPI, Official API to pull data
## Original source code on github: https://github.com/codingforentrepreneurs/30-Days-of-Python/tree/master/tutorial-reference/Day%2019
## Credit: codingforentrepreneurs


class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

In [347]:
client_id = 'client_secret = '
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [348]:
spotify = SpotifyAPI(client_id,client_secret)
artist_ids=[]
artist_search=artists_for_analysis
for i in range(0,len(artist_search)):
    d2=spotify.search({"artist":artist_search[i]},search_type="track")
    artist_ids.append(d2['tracks']['items'][0]['artists'][0]['uri'][15:])

print(artist_ids)

q=artist%3AMegan+Thee+Stallion&type=track
q=artist%3ABenee&type=track
q=artist%3ALil+Nas+X&type=track
q=artist%3ABTS&type=track
q=artist%3ADoja+Cat&type=track
q=artist%3AOlivia+Rodrigo&type=track
['181bsRPaVXVlUKXrxwZfHK', '0Cp8WN4V8Tu4QJQwCN5Md4', '7jVv8c5Fj3E9VhNjxT4snq', '3Nrfpe0tUJi4K4DXYWgMUX', '5cj0lLjcoR7YOSnhnX0Po5', '1McMsnEElThX1knmY4oliG']


In [349]:
#the id here is the artist's Spotify id
albums_info=[]
artists=artist_ids
for i in range(0,len(artists)):
    albums_info.append(sp.artist_albums(artists[i]))

album_names=[]
album_ids=[]
for k in range(0,len(albums_info)):
    for l in range(0,len(albums_info[k]['items'])):
        album_names.append(albums_info[k]['items'][l]['name'])
        album_ids.append(albums_info[k]['items'][l]['id'])

albums_final=np.array([album_names,album_ids]).T.tolist()

In [350]:
len(albums_final)
#print(albums_final)

120

In [351]:
#album_songs=[]
album_songs_name=[]
album_songs_id=[]
album_ids=[]
album_ids_new=[]
#Loop through to get the album ids for each album
for k in range(0,len(albums_final)):
    album_ids.append(albums_final[k][1])
    
for j in range(0,len(album_ids)):
    test=sp.album_tracks(album_ids[j])
    for i in range(0,len(test['items'])):
        album_songs_name.append(test['items'][i]['name'])
        album_songs_id.append(test['items'][i]['id'])
        
albums_songs_f=np.array([album_songs_name,album_songs_id]).T.tolist()
#print(len(album_songs_final))
#print(album_songs_final)
#print(np.array([album_songs_name,album_songs_id]).T.tolist())

In [352]:
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  if features == [None]:
    acousticness = 0
    danceability = 0
    energy = 0
    instrumentalness = 0
    liveness = 0
    loudness = 0
    speechiness = 0
    tempo = 0
    time_signature = 0
  else:  
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

  track = [name, album, artist, release_date, length, popularity, acousticness
           , danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

In [353]:
songs_details=[]
for i in range(0,len(albums_songs_f)):
    songs_details.append(getTrackFeatures(albums_songs_f[i][1]))

df = pd.DataFrame(songs_details,columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity'
                                           , 'acousticness', 'danceability', 'energy', 'instrumentalness'
                                           , 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])



In [354]:
df

,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,Tuned In Freestyle,Something for Thee Hotties,Megan Thee Stallion,2021-10-29,123636,66,0.000108,0.850,0.836,0.000115,0.1390,-6.689,0.0789,131.961,4
1,Megan Monday Freestyle,Something for Thee Hotties,Megan Thee Stallion,2021-10-29,113684,65,0.005270,0.884,0.615,0.000000,0.2710,-5.352,0.1940,152.052,4
2,Trippy Skit (feat. Juicy J),Something for Thee Hotties,Megan Thee Stallion,2021-10-29,72520,60,0.865000,0.601,0.181,0.000000,0.3510,-19.144,0.8220,99.153,5
3,Southside Forever Freestyle,Something for Thee Hotties,Megan Thee Stallion,2021-10-29,177230,63,0.001510,0.940,0.357,0.000084,0.3560,-6.693,0.1510,129.951,4
4,Outta Town Freestyle,Something for Thee Hotties,Megan Thee Stallion,2021-10-29,80816,63,0.007130,0.689,0.675,0.027900,0.1110,-8.303,0.0845,196.033,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,"Stick to the Status Quo - From ""High School Mu...","Just for a Moment (From ""High School Musical: ...",Olivia Rodrigo,2020-01-01,117400,39,0.003510,0.405,0.778,0.000000,0.1350,-4.884,0.0447,159.894,4
909,"Out of the Old - From ""High School Musical: Th...","Out of the Old (From ""High School Musical: The...",Olivia Rodrigo,2019-12-18,168688,51,0.560000,0.624,0.621,0.000000,0.1330,-6.756,0.1190,113.011,4
910,"All I Want - From ""High School Musical: The Mu...","All I Want (From ""High School Musical: The Mus...",Olivia Rodrigo,2019-11-27,177322,75,0.090200,0.376,0.430,0.000000,0.0912,-6.585,0.0328,77.599,3
911,"What I've Been Looking For - From ""High School...","All I Want (From ""High School Musical: The Mus...",Olivia Rodrigo,2019-11-27,125096,48,0.827000,0.734,0.291,0.000000,0.1440,-7.921,0.0418,135.779,4


In [355]:
df = df.drop(df[(df.artist != artists_for_analysis[0]) 
                & (df.artist != artists_for_analysis[1]) 
                & (df.artist != artists_for_analysis[2])
                & (df.artist != artists_for_analysis[3])
                & (df.artist != artists_for_analysis[4])
                & (df.artist != artists_for_analysis[5])].index)
df=df.rename(columns=df.iloc[0]).drop(df.index[0])

In [356]:
songs_details=[]
songs_details=df.values.tolist() 
     
# Printing the final list

In [357]:
###Connection to Google Sheets###

In [358]:
pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [359]:
pip install oauth2client

Note: you may need to restart the kernel to use updated packages.


In [360]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint

In [361]:
scope = ["https://spreadsheets.google.com/feeds"
         ,'https://www.googleapis.com/auth/spreadsheets'
         ,"https://www.googleapis.com/auth/drive.file"
         ,"https://www.googleapis.com/auth/drive"]

In [362]:
creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)

In [363]:
client = gspread.authorize(creds)

In [364]:
sheet = client.open("SpotifyPythonData-PythonAPI").sheet1

In [365]:
sheet.batch_clear(["A2:V"])

{'spreadsheetId': '1QBEdAv6rnyyjGt4kVbdW8Ue8bcCSIyVxMtrv7nj-5vk',
 'clearedRanges': ['spotify_artists!A2:V28060']}

In [368]:
insertRow = songs_details
sheet.insert_rows(insertRow,2)

{'spreadsheetId': '1QBEdAv6rnyyjGt4kVbdW8Ue8bcCSIyVxMtrv7nj-5vk',
 'updates': {'spreadsheetId': '1QBEdAv6rnyyjGt4kVbdW8Ue8bcCSIyVxMtrv7nj-5vk',
  'updatedRange': 'spotify_artists!A2:O730',
  'updatedRows': 729,
  'updatedColumns': 15,
  'updatedCells': 10935}}

In [369]:
print("Code executed, refresh the dashboard")

Code executed, refresh the dashboard
